## Información no Estructurada
# Práctica 3.3 &mdash; Búsqueda Web y Recomendación - Sistemas de Recomendación
### Autores: Íñigo Gómez Carvajal y Jon Zorrilla Gamboa 

# 0.  Ejercicios

## Ejemplo recomendación aleatoria

Leer un conjunto de datos público

In [ ]:
dataset_urls=[
    'http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip',
    'http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip',
    'http://files.grouplens.org/datasets/hetrec2011/hetrec2011-delicious-2k.zip'
]

# We will work with the first dataset
dataset_url=dataset_urls[0]
dataset_name=dataset_url[dataset_url.rfind('/')+1:]
from pathlib import Path
dataset_file = Path(dataset_name)
if not dataset_file.exists():
    # Let's download it
    import requests
    r = requests.get(dataset_url, allow_redirects=True)
    open(dataset_name, 'wb').write(r.content)
else:
    print('Dataset ', dataset_name, ' will not be downloaded because it already exists')

import zipfile
import pandas as pd

# Let's first inspect which files we have in the dataset
print('Files in ', dataset_name)
with zipfile.ZipFile(dataset_name) as z:
    for name in z.namelist():
        print(name)
print()
# We use the file that contains user, items, and their ratings
target_file='user_ratedmovies-timestamps.dat'
print('Initial content in ', target_file)
with zipfile.ZipFile(dataset_name) as z:
   with z.open(target_file) as f:
      data = pd.read_csv(f, header=0, delimiter="\t")
      print(data.head())    # print the first 5 rows
# we also need the names of the movies:
target_file='movies.dat'
print('Initial content in ', target_file)
with zipfile.ZipFile(dataset_name) as z:
   with z.open(target_file) as f:
        movies = pd.read_csv(f, header=0, delimiter="\t", encoding = "ISO-8859-1")
        movies = movies[['id', 'title']]
        print(movies.head())    # print the first 5 rows

Files in  hetrec2011-movielens-2k-v2.zip
user_taggedmovies-timestamps.dat
movie_actors.dat
movie_countries.dat
movie_directors.dat
movie_genres.dat
movie_locations.dat
movie_tags.dat
movies.dat
readme.txt
tags.dat
user_ratedmovies.dat
user_ratedmovies-timestamps.dat
user_taggedmovies.dat

Initial content in  user_ratedmovies-timestamps.dat
   userID  movieID  rating      timestamp
0      75        3     1.0  1162160236000
1      75       32     4.5  1162160624000
2      75      110     4.0  1162161008000
3      75      160     2.0  1162160212000
4      75      163     4.0  1162160970000
Initial content in  movies.dat
   id                        title
0   1                    Toy story
1   2                      Jumanji
2   3               Grumpy Old Men
3   4            Waiting to Exhale
4   5  Father of the Bride Part II


Generar recomendaciones aleatorias

In [ ]:
for _ in range(5):
    id=data.sample()['movieID'].iloc[0]
    print(movies.loc[movies['id']==id]['title'].iloc[0])

As Good as It Gets
Edward Scissorhands
Hoodwinked!
Jerry Maguire
Gladiator


## Ejemplo recomendación popular

In [ ]:
pop_counts=data.groupby('movieID')['movieID'].count().sort_values(ascending=False).reset_index(name='pop')
#print(pop_counts.head())
for i in range(5):
    id=pop_counts.loc[i, 'movieID']
    print(movies.loc[movies['id']==id]['title'].iloc[0])
print()
#print(pop_counts.tail())
for i in range(5):
    id=pop_counts.loc[len(pop_counts)-1-i, 'movieID']
    print(movies.loc[movies['id']==id]['title'].iloc[0])

The Matrix
The Lord of the Rings: The Fellowship of the Ring
Forrest Gump
Pulp Fiction
The Lord of the Rings: The Two Towers

Dakota
Texas Terror
Darfur Now
Ice-Cold in Alex
Kansas City Confidential


# 1.  Práctica

En esta práctica se usará un dataset (dividido en entrenamiento y test) que se obtendrá de la siguiente manera, una vez que hayáis rellenado vuestros gustos por las películas que se indican en la siguiente hoja de cálculo:
https://docs.google.com/spreadsheets/d/1YmCY-T0ByZRXz8DpaFyieGfsS2jvxsprZr5dsSh8mpE/edit?usp=sharing

Una vez completo, en las pestañas *ratings_training* y *ratings_test* se encontrarán los datos (tripletas <usuario, item, rating>) a utilizar en el resto de la práctica.

Observación: usamos DataFrames por comodidad, pero implementaciones más eficientes podrían hacer uso de las [matrices dispersas](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) de SciPy u otras estructuras de datos que consideréis convenientes.

In [ ]:
import pandas as pd

SHEET_ID = '1YmCY-T0ByZRXz8DpaFyieGfsS2jvxsprZr5dsSh8mpE'
SHEET_NAME = 'ratings_training'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_training = pd.read_csv(url)
print(df_training.head())
print(df_training.describe())
SHEET_NAME = 'ratings_test'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_testing = pd.read_csv(url)
print(df_testing.head())

   user  item  rating
0     0   148     4.0
1     0    72     4.0
2     0   143     5.0
3     0    37     4.0
4     0    98     3.0
             user        item      rating
count  905.000000  905.000000  905.000000
mean     6.436464   73.879558    3.745856
std      4.285006   43.401346    0.940748
min      0.000000    1.000000    1.000000
25%      2.000000   37.000000    3.000000
50%      7.000000   72.000000    4.000000
75%     10.000000  113.000000    4.500000
max     13.000000  150.000000    5.000000
   user  item  rating
0     0    86     4.0
1     0   123     3.0
2     0    96     4.0
3     0   137     3.5
4     0    73     4.0


## 1.1 Implementar algoritmo de recomendación

Aplicar un algoritmo de filtrado colaborativo basado en memoria para generar recomendaciones de películas para cada estudiante. La estrategia a usar será a elección de cada estudiante, teniendo en cuenta las consideraciones vistas en clase. Aplicarlo utilizando las puntuaciones de entrenamiento como base para recomendar películas del conjunto de test, y observar el nivel de acierto (visualmente, comparando puntuación real de test y predicción del recomendador).

Para implementar este algoritmo, haremos uso de la regla del coseno para estudiar la similitud entre los usuarios. Para ello, definiremos 5 funciones donde cada una tiene una de las siguientes funciones:
- A partir de los datos de entrenamiento, construimos una matriz usuario-película.
- Calculamos la similitud entre los usuarios mediante la relación coseno.
- Encontramos los k usuarios más parecidos.
- Calculamos la puntuación de las películas.
- Recomendamos las películas con mayor puntuación.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def build_user_movie_matrix(train_data, num_users, num_movies):
    user_movie_matrix = np.zeros((num_users, num_movies))
    for user, movie, rating in (train_data.iloc[i] for i in range(len(train_data))):
        user_movie_matrix[int(user)-1][int(movie)-1] = rating
    return user_movie_matrix

def calculate_similarity(user_movie_matrix):
    similarity_matrix = cosine_similarity(user_movie_matrix)
    return similarity_matrix

def find_similar_users(similarity_matrix, user_id, k):
    similar_users = np.argsort(similarity_matrix[int(user_id)-1])[::-1][1:k+1]
    return similar_users

def calculate_predicted_rating(user_movie_matrix, similarity_matrix, user_id, movie_id, similar_users):
    numerator = 0
    denominator = 0
    for user in similar_users:
        if user_movie_matrix[user][movie_id-1] != 0:
            numerator += similarity_matrix[int(user_id)-1][user] * user_movie_matrix[user][movie_id-1]
            denominator += similarity_matrix[int(user_id)-1][user]
    if denominator == 0:
        return 0
    else:
        return numerator / denominator

def recommend_movies(train_data, test_data, num_users, num_movies, k):
    user_movie_matrix = build_user_movie_matrix(train_data, num_users, num_movies)
    similarity_matrix = calculate_similarity(user_movie_matrix)
    recommendations = {}
    res = {}
    for user, movie, rating in (test_data.iloc[i] for i in range(len(test_data))):
        if user not in recommendations:
            similar_users = find_similar_users(similarity_matrix, user, k)
            recommendations[user] = {}
            for i in range(num_movies):
                if user_movie_matrix[int(user)-1][i] == 0:
                    predicted_rating = calculate_predicted_rating(user_movie_matrix, similarity_matrix, user, i+1, similar_users)
                    recommendations[user][i+1] = predicted_rating
            top_recommendations = sorted(recommendations[user].items(), key=lambda x:x[1], reverse=True)[:10]
            if user not in res:
              res[user] = top_recommendations
            else:
              res[user].append(top_recommendations)
    return res

In [ ]:
recommend_movies(df_training, df_testing, 14, 150, 3)

{0.0: [(8, 5.0),
  (64, 4.84383860686743),
  (126, 4.7650345738650035),
  (119, 4.5),
  (58, 4.491547046667619),
  (73, 4.491547046667619),
  (124, 4.352291560199812),
  (144, 4.258901716363702),
  (17, 4.167692826767524),
  (12, 4.0)],
 1.0: [(125, 4.730307367743689),
  (21, 4.699728669394203),
  (36, 4.648378745606145),
  (8, 4.539385264512623),
  (121, 4.5368889979188225),
  (140, 4.497489141472114),
  (35, 4.399457338788404),
  (88, 4.351621254393856),
  (112, 4.246233712208171),
  (56, 4.0)],
 10.0: [(6, 5.0),
  (35, 5.0),
  (43, 5.0),
  (65, 5.0),
  (73, 5.0),
  (100, 5.0),
  (106, 5.0),
  (15, 4.500521619187528),
  (66, 4.5),
  (114, 4.5)],
 11.0: [(126, 4.748950988406431),
  (119, 4.5),
  (63, 4.262273547159652),
  (132, 4.262273547159652),
  (120, 4.143118171757753),
  (121, 4.0449073932543556),
  (89, 4.0),
  (93, 4.0),
  (102, 4.0),
  (146, 4.0)],
 12.0: [(35, 5.0),
  (88, 5.0),
  (39, 4.837445117643706),
  (64, 4.837445117643706),
  (6, 4.75380818637044),
  (106, 4.75380818

En este caso, se ha realizado el algoritmo para k=3 vecinos próximos; es decir, se han realizado las comparaciones teniendo en cuenta únicamente los 3 usuarios con mayor similitud a cada uno de los usuarios. 

Como se puede ver, se han imprimido las 10 mejores recomendaciones para cada usuario, basándonos en los 3 usuarios más cercanos a cada uno de los usuarios, y ordenados en orden decreciente, empezando por la película con mayor rating.

## 1.2 Evaluación

Calcular el MAE y RMSE de las recomendaciones producidas. Contrastar los resultados (MAE y RMSE) de dos estrategias diferentes de recomendación (o de la misma estrategia con diferentes parámetros).

Calcular P@N (precisión) y R@N (recall) de las recomendaciones producidas a distintos N, considerando como relevantes aquellos ítems con un rating mayor que 3 en test. Para ello, tened en cuenta que habrá que predecir una puntuación (para cada usuario) usando todos los ítems del sistema excepto aquellos que ya ha visto en entrenamiento, ordenando dichas puntuaciones para generar el ranking.

In [ ]:
def calculate_mae_rmse(results, df_testing):
  mae = 0
  rmse = 0
  count = 0
  for user in results.keys():
    for movie, rating in results[user]:
      try:
        rate_test = df_testing[(df_testing["user"]==user) & (df_testing["item"]==movie)]["rating"].iloc[0]
        mae += abs(rating - rate_test)
        rmse += (rating - rate_test)**2
        count += 1
      except IndexError:
        pass
  
  mae = mae/count
  rmse = np.sqrt(rmse/count)
  return(mae, rmse)

In [ ]:
results = recommend_movies(df_training, df_testing, 14, 150, 3)
calculate_mae_rmse(results, df_testing)

(0.7019482871052402, 0.9372845113338175)

Hemos hallado el mae y el rmse comprobando el resultado que nos ha producido el algoritmo con el conjunto de datos de test. Como hay ciertas películas que no aparecen en el conjunto de datos de test, no hemos tenido esas en cuenta a la hora de dividir el conteo para el mae y para el rmse. En los resultados obtenidos se puede observar como no nos hemos equivocado más de 1 punto para los ratings; es decir, el error relativo es menor al 20%. Estos resultados no están mal si tenemos en cuenta que el conjunto de datos de entrenamiento no es muy grande, y que además hay bastante sesgo ya que muchos usuarios no habían visto muchas de las películas presentes en el conjunto de datos. 

Ahora, calcularemos el mae y el rmse para la misma estrategia con otros parámetros. En este caso, haremos uso de la similitud de entre un usuario y todos los usuarios como vecinos próximos para estudiar su influencia.

In [ ]:
for i in range(1,13):
  results = recommend_movies(df_training, df_testing, 14, 150, i)
  print(f"Errores mae y rmse:{calculate_mae_rmse(results, df_testing)}, para n_vecinos={i}" )

Errores mae y rmse:(0.7950819672131147, 1.034249549744906), para n_vecinos=1
Errores mae y rmse:(0.7267707614036512, 0.9601712594250206), para n_vecinos=2
Errores mae y rmse:(0.7019482871052402, 0.9372845113338175), para n_vecinos=3
Errores mae y rmse:(0.6671116017606855, 0.8218335186821566), para n_vecinos=4
Errores mae y rmse:(0.6208656381383658, 0.7708905167534325), para n_vecinos=5
Errores mae y rmse:(0.5981532797221947, 0.7279121895848298), para n_vecinos=6
Errores mae y rmse:(0.592854500955739, 0.7108939861579296), para n_vecinos=7
Errores mae y rmse:(0.6891704863672369, 0.847165821762154), para n_vecinos=8
Errores mae y rmse:(0.6119477043287916, 0.774695557322029), para n_vecinos=9
Errores mae y rmse:(0.6016097479505402, 0.7757363605640005), para n_vecinos=10
Errores mae y rmse:(0.6021138685289438, 0.7749682226453529), para n_vecinos=11
Errores mae y rmse:(0.6142819110284283, 0.7673827831938693), para n_vecinos=12


Como se puede observar en estos resultados, el error empieza siendo máximo para un vecino próximo, empieza a disminuir al aumentar el número de vecinos pero luego vuelve aumentar a partir de 8 vecinos próximos, tanto para el mae como para el rmse, obteniendo el mínimo error para 7 vecinos. Este resultado se puede interpretar como que a partir de 8 vecinos, el resultado empeora porque realmente no se están teniendo en cuenta usuarios con los mismos gustos; y se empiezan a hacer comparaciones con usuarios que han puntuado de manera muy diferente cada una de las películas.

## 1.3 Informe

Escribir un informe explicando los resultados y las técnicas concretas aplicadas.

Para esta práctica, se ha implementado un algoritmo de recomendación de películas, aplicado al dataset de películas que se ha puntuado en clase. 

En primer lugar, se lee el dataset de películas y se hace un pequeño análisis sobre él. Luego, para implementar el algoritmo de recomendación, se definen las siguientes funciones:
- `build_user_movie_matrix`: Mediante esta función, se construye una matriz que relaciona los usuarios con las películas.
- `calculate_similarity`: Mediante esta función, se calcula la similaridad mediante la función `cosine_similarity`de sklearn, para poder obtener la similaridad entre los usuarios en función de las películas de la matriz que se obtiene de la función anterior.
- `find_similar_users`: Mediante esta función, se obtiene los usuarios más similares según el criterio del coseno previamente definido. 
- `calculate_predicted_rating`: Esta función calcula el rating que se predice para cada película, para cada usuario, haciendo uso de la matriz de similitud de la segunda función y la matriz usuario-película de la primera función.
- `recommend_movies`: Por último, esta función ordena los usuarios en función de la similitud entre ellos previamente hallada, y predice la nota de las películas previamente calculada por la función `calculate_predicted_rating`. Devuelve un diccionario donde para cada usuario como llave, tiene asociado una tupla (película, nota predecida). Esta función también tiene una variable extra `k`, la cual se puede modificar para obtener la similitud de cada usuario con los `k` vecinos más cercanos.

Después, hemos definido una función que calcula el mae y el rmse. Teniendo en cuenta las películas presentes en el conjunto de películas de test y las predicciones obtenidas previamente. Estos errores, dan, en general, menos de un 20% de error en las notas predecidas. 

Por último, se calculan los errores mae y rmse para todos los valores posibles de vecinos próximos, desde 1 hasta 13 vecinos. Como se puede observar, el error es máximo cuando se compara únicamente con un vecino, empieza a disminuir hasta comparar con 8 vecinos y luego vuelve a subir. 

## Opcional A

Utilizar librerías (por ejemplo, Surprise o LightFM en Python) sobre los mismos datos para generar las recomendaciones y comparar con los resultados anteriores. Contrastar los valores de MAE y RMSE de las diferentes pruebas. Para cualquier duda o dificultad con este apartado consultar con el profesor.

In [ ]:
# pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095401 sha256=9a55d035639e98325dffd371e94486a6978a5ba00339cc535830dfe6665a65ea
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.prediction_algorithms.knns import KNNBasic

In [ ]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
trainset = Dataset.load_from_df(df_training, reader).build_full_trainset()
testset = Dataset.load_from_df(
    df_testing, reader).build_full_trainset().build_testset()

In [ ]:
cosine_sim = {
    'name': 'cosine',
    'user_based': False 
}

algorithm = [("Cosine", KNNBasic(k=20, sim_options=cosine_sim))]

for name, alg in algorithm:
    # Train the algorithm on the trainset, and predict ratings for the testset
    alg.fit(trainset)
    print(f"{name}")
    predictions = alg.test(testset)

    # Then compute error and evaluation metrics
    mae = accuracy.mae(predictions)
    rmse = accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Cosine
MAE:  0.6974
RMSE: 0.9263


## Opcional B

Implementar un algoritmo de recomendación mediante factorización de matrices.

In [ ]:
# TODO

## Opcional C

Implementar un algoritmo basado en contenido (usando los géneros de las películas como atributos).

In [ ]:
# TODO

## Opcional D

Repetir alguno de los ejercicios (tanto el 1.2 como los opcionales A, B o C) con alguna colección de prueba (p.e., [MovieLens](https://grouplens.org/datasets/movielens/)).

In [ ]:
from surprise import SVD
from surprise.model_selection import cross_validate

movielens = Dataset.load_builtin('ml-100k')

for name, alg in algorithm:
    print(f"--- {name} --- ")
    # Run 5-fold cross-validation and print results
    cross_validate(alg, movielens, n_jobs=-1, 
                   measures=['MAE', 'RMSE'], cv=5, 
                   verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
--- Cosine --- 
Evaluating MAE, RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8296  0.8338  0.8349  0.8281  0.8234  0.8300  0.0041  
RMSE (testset)    1.0474  1.0534  1.0596  1.0451  1.0396  1.0490  0.0069  
Fit time          1.15    1.13    1.58    1.18    0.66    1.14    0.29    
Test time         5.65    6.26    4.47    4.48    2.26    4.62    1.37    
